# Merge Crime Data with Streetlights

For the distances, I'm assuming that because everything is within the Boston area that we can ignore the curvature of the Earth and just treat distances as direct.

The radius I chose for streetlight "neighborhood" is 0.003 in long lat euclidian distance, and is based off of inspection of a map with interactive Long lat coordinates, and i figured it out by playing with a map of boston to determine what was resonable. I was looking to capture enough area to get a sense of the streetlamp density without making it so big that it would'nt be informative

In [1]:
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd
import numpy as np
from scipy.spatial.distance import cdist
from shapely.geometry import Point, Polygon
warnings.filterwarnings('ignore')

In [2]:
# read in crimes dataframe without index
crime_incidents = pd.read_csv('data/crime_incidents.csv') 
crime_incidents = crime_incidents.drop(crime_incidents.columns[0], axis=1)

# read in weather data without index
streetlights = pd.read_csv('data/streetlights.csv')
streetlights = streetlights.drop(streetlights.columns[0], axis=1)

In [3]:
# extract only latitude and longitude from crimes
crime_coords = crime_incidents[['Lat','Long']]

In [ ]:
# get crime and streetlight coordinates into numpy arrays
X = np.array(crime_coords).reshape(-1,2)
Y = np.array(streetlights).reshape(-1,2)

# set up streetlamp density radius
radius = 0.01

# append min dist and lamp density to crimes dataframe
for i,row in enumerate(X):
    
    # calculate distance from crime to all street lamps
    dists = cdist(row.reshape(1,-1), Y)[0]
    
    # append min streetlamp distance and lamp density in the neighborhood to crimes
    crime_incidents.at[i,'lamp_min_dist'] = np.min(dists)
    crime_incidents.at[i,'lamp_density']  = len(dists[dists < radius])

In [ ]:
crime_incidents.to_csv('data/crimes_with_lamps.csv')